In [ ]:
import os
import time
from datetime import timedelta
import json
import scipy
import numpy as np
import pandas as pd
from PIL import Image
import torch
import autogen
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_recall_curve,average_precision_score
from sklearn.linear_model import LinearRegression

import sys
sys.path.append('..')
sys.path.append('../src')
import src.inference as src_inference
import src.finetune as src_finetune

import seaborn as sns
sns.set(context='paper', style='white')
sns.set_color_codes()
import matplotlib.pyplot as plt
# %matplotlib inline

import warnings
warnings.filterwarnings('ignore')

### 1.Inference Function

In [ ]:
def Analyze_Protein_Property(file_path, task_type, finetune=False):
    task_type = task_type.lower()
    if not finetune:
        return src_inference.launch(file_path, task_type)
    else:
        return src_finetune.launch(file_path, task_type)
    
def Analyze_Protein_Drug_Interaction(file_path, task_type, finetune=False):
    task_type = task_type.lower()
    if not finetune:
        return src_inference.launch(file_path, task_type)
    else:
        return src_finetune.launch(file_path, task_type)
    
def Analyze_Protein_Protein_Interaction(file_path, task_type, finetune=False):
    task_type = task_type.lower()
    if not finetune:
        return src_inference.launch(file_path, task_type)
    else:
        return src_finetune.launch(file_path, task_type)

### 2.Evaluate Function

In [ ]:
def reg_metrics(labels, preds):
    labels = np.array(labels) if isinstance(labels, list) else labels
    preds = np.array(preds) if isinstance(preds, list) else preds
    rmse = ((preds - labels) ** 2).mean() ** 0.5
    mae = (np.abs(preds - labels)).mean()
    corr = scipy.stats.pearsonr(preds, labels)
    lr = LinearRegression()
    lr.fit(preds.reshape(-1, 1), labels)
    y_ = lr.predict(preds.reshape(-1, 1))
    sd = (((labels - y_) ** 2).sum() / (len(labels) - 1)) ** 0.5
    return rmse, mae, corr, sd
            
def Evaluate(inference_file_path, task_type):
    task_type = task_type.lower()
    inference_dict = json.load(open(inference_file_path, 'r'))
    metrics_dict = {}
    if task_type in ['stability', 'fluorescence', 'pdbbind']:
        for key in inference_dict.keys():
            if isinstance(inference_dict[key], list):
                labels = [label for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                preds = [pred for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                rmse, mae, corr, sd = reg_metrics(labels, preds)
                metrics_dict[key] = {'rmse': rmse, 'mae': mae, 'corr': corr, 'sd': sd, 'label': labels, 'pred': preds}
    elif 'remote' in task_type or 'homology' in task_type:
        for key in inference_dict.keys():
            if isinstance(inference_dict[key], list):
                labels = [label for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                preds = [pred for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                accuracy = accuracy_score(labels, preds)
                metrics_dict[key] = {'accuracy': accuracy}
    elif 'secondary' in task_type or 'structure' in task_type:
        for key in inference_dict.keys():
            if isinstance(inference_dict[key], list):
                labels = [label for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                preds = [pred for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                labels, preds = sum(labels, []), sum(preds, [])
                accuracy = accuracy_score(labels, preds)
                metrics_dict[key] = {'accuracy': accuracy}
    elif task_type in ['kinase'] or ('antigen' in task_type or 'binding' in task_type):
        for key in inference_dict.keys():
            if isinstance(inference_dict[key], list):
                labels = [label for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                preds = [pred for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                fpr, tpr, thersholds = roc_curve(labels, preds)
                roc_auc = auc(fpr, tpr)
                precision, recall, _ = precision_recall_curve(labels, preds)
                prc = average_precision_score(labels, preds)
                metrics_dict[key] = {'fpr': fpr.tolist(), 'tpr': tpr.tolist(), 'roc_auc': roc_auc, \
                                    'precision': precision.tolist(), 'recall': recall.tolist(), 'prc': prc}
    elif task_type in ['skempi']:
        for key in inference_dict.keys():
            if isinstance(inference_dict[key], list):
                labels = [label for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                preds = [pred for _ , (pro_id, label, pred) in enumerate(inference_dict[key])]
                rmse, mae, corr, sd = reg_metrics(labels, preds)
                metrics_dict[key] = {'rmse': rmse, 'mae': mae, 'corr': corr, 'sd': sd, 'label': labels, 'pred': preds}
            elif isinstance(inference_dict[key], dict):
                metrics_dict[key] = {}
                for sub_key in results[key]:
                    labels = [label for _ , (pro_id, label, pred) in enumerate(inference_dict[key][sub_key])]
                    preds = [pred for _ , (pro_id, label, pred) in enumerate(inference_dict[key][sub_key])]
                    rmse, mae, corr, sd = reg_metrics(labels, preds)
                    metrics_dict[key][sub_key] = {'rmse': rmse, 'mae': mae, 'corr': corr, 'sd': sd, 'label': labels, 'pred': preds}
    
    metrics_results_dir = "metrics_results"
    os.makedirs(metrics_results_dir, exist_ok=True)
    with open(os.path.join(metrics_results_dir, f"{task_type}_metrics.json"), 'w') as f:
        json.dump(metrics_dict, f, indent=4)
    f.close()
    
    return os.path.join(metrics_results_dir, f"{task_type}_metrics.json")

### 3.Visualize Function

In [ ]:
def combined_image(image_files, save_path):
    width, height = Image.open(image_files[0]).size
    total_width = width * len(image_files)
    max_height = height  
    new_im = Image.new('RGB', (total_width, max_height))
    x_offset = 0  
    for img_path in image_files:
        img = Image.open(img_path)
        new_im.paste(img, (x_offset, 0))
        x_offset += img.width
    new_im.save(save_path)
    for img_path in image_files:
        os.remove(img_path)
    
def plot_reg(visualization_results_dir, metrics_dict, set_colors, task_type, min_value, max_value, interval):
    plt.rcParams['font.size'] = 11
    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    
    image_files = []
    for key, metrics in metrics_dict.items():
        set_name = key.capitalize() + (' set (core2016)' if task_type == 'pdbbind' and key == 'test' else ' set')
        rmse, mae, corr, sd = metrics['rmse'], metrics['mae'], metrics['corr'], metrics['sd']
        table = pd.DataFrame()
        table['real'] = metrics['label']
        table['predicted'] = metrics['pred']
        
        grid = sns.jointplot('real', 'predicted', data=table, kind='scatter', color=set_colors[set_name],
                             space=0, size=4, ratio=4, s=20, edgecolor='black')
        grid.ax_joint.set_xticks(np.arange(min_value, max_value, interval))
        grid.ax_joint.set_yticks(np.arange(min_value, max_value, interval))
        grid.set_axis_labels(xlabel='real', ylabel='predicted', fontsize=12)
        grid.fig.suptitle(task_type.title(), fontsize=14)
        grid.fig.subplots_adjust(top=0.92)

        if task_type in ['stability', 'fluorescence']:
            grid.ax_joint.text(-1.8, 3.5, set_name, fontsize=13)
            grid.ax_joint.text(4.3, 5.6, 'Pearson: %.2f ' % corr[0])
            grid.ax_joint.text(4.3, 5.1, 'RMSE: %.2f' % (rmse))   
            grid.ax_joint.text(4.3, 4.6, 'MAE: %.2f' % (mae))
            grid.ax_joint.text(4.3, 4.1, 'SD: %.2f ' % sd)
        elif task_type in ['pdbbind']:
            grid.ax_joint.text(1, 14, set_name, fontsize=13)
            grid.ax_joint.text(16, 19.5, 'Pearson: %.2f ' % corr[0])   
            grid.ax_joint.text(16, 18.5, 'RMSE: %.2f' % (rmse))   
            grid.ax_joint.text(16, 17.5, 'MAE: %.2f' % (mae))
            grid.ax_joint.text(16, 16.5, 'SD: %.2f ' % sd)
    
        sub_image_file = os.path.join(visualization_results_dir, f'{task_type}_{key}_visualization.jpg')
        image_files.append(sub_image_file)
        grid.fig.savefig(sub_image_file, bbox_inches='tight', dpi=1024)

    combined_image_file = os.path.join(visualization_results_dir, f'{task_type}_visualization.jpg')
    combined_image(image_files, combined_image_file)
    return combined_image_file


def plot_cls(visualization_results_dir, metrics_dict, alternative_colors, task_type):

    plt.rcParams['font.size'] = 12
    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False

    categories, accuracies, colors = [], [], []
    for i, (key, metrics) in enumerate(metrics_dict.items()):
        categories.append(key.capitalize())
        accuracies.append(metrics['accuracy'])
        colors.append(alternative_colors[i])

    plt.subplots(figsize=(7,4))
    bars = plt.bar(categories, accuracies, color=colors)
    plt.xticks(categories, rotation=30, fontsize=13)
    plt.yticks(np.arange(0, 0.61, 0.1), fontsize=12)
    plt.ylim(0, 0.6)
    plt.tick_params(axis='both', direction='in', length=6, width=2)
    plt.grid(True, axis='y')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 3), ha='center', va='bottom', fontsize=13.5)
    plt.ylabel('Accuracy', fontsize=13)
    plt.title(task_type.title(), fontsize=14)
    plt.tight_layout()
    image_file = os.path.join(visualization_results_dir, f'{task_type}_visualization.jpg')
    plt.savefig(image_file, bbox_inches='tight', dpi=1024)

    return image_file

def plot_roc_curve(visualization_results_dir, metrics_dict, alternative_colors, task_type):
    plt.rcParams['font.size'] = 12
    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.subplots(figsize=(7, 6))
     
    for i, (key, metrics) in enumerate(metrics_dict.items()):
        fpr, tpr, roc_auc = metrics['fpr'], metrics['tpr'], metrics['roc_auc']
        plt.plot(fpr, tpr, color=alternative_colors[i], lw=2, label='%s (AUC = %0.2f)' % (key.capitalize(), roc_auc))       
    plt.plot([0, 1], [0, 1], color='grey', lw=2, label='Random Guess',linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=12)  
    plt.yticks(fontsize=12)  
    current_yticks = plt.gca().get_yticks() 
    for y in current_yticks:
        plt.axhline(y=y, color='grey', linestyle='-', lw=0.77, alpha=0.37) 
    plt.xlabel('False Positive Rate',fontsize=13)
    plt.ylabel('True Positive Rate',fontsize=13)
    plt.title('ROC curves on Kinase',fontsize=14)
    plt.legend(loc="lower right",fontsize=14)
    plt.tight_layout()
    image_file = os.path.join(visualization_results_dir, f'{task_type}_roc_curve_visualization.jpg')
    plt.savefig(image_file, bbox_inches='tight', dpi=1024)
    
    return image_file

def plot_prc_curve(visualization_results_dir, metrics_dict, alternative_colors, task_type):
    plt.rcParams['font.size'] = 12
    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.subplots(figsize=(7,6))
    
    for i, (key, metrics) in enumerate(metrics_dict.items()):
        recall, precision, prc = metrics['recall'], metrics['precision'], metrics['prc']
        plt.plot(recall, precision, color=alternative_colors[i], lw=2, label='%s (PRC = %0.2f)' % (key.capitalize(), prc))
    
    plt.plot([0, 1], [1, 0], color='grey', lw=2, label='Random Guess',linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=12)  
    plt.yticks(fontsize=12)
    current_yticks = plt.gca().get_yticks()
    for y in current_yticks:
        plt.axhline(y=y, color='grey', linestyle='-', lw=0.77, alpha=0.37)
    plt.xlabel('Recall',fontsize=13)
    plt.ylabel('Precision',fontsize=13)
    plt.title('Precision-Recall curves on Kinase',fontsize=14)
    plt.legend(loc='lower right',fontsize=14)
    plt.tight_layout()
    image_file = os.path.join(visualization_results_dir, f'{task_type}_prc_curve_visualization.jpg')
    plt.savefig(image_file, bbox_inches='tight', dpi=1024)
    
    return image_file


def plot_binarycls(visualization_results_dir, metrics_dict, alternative_colors, task_type):
    roc_image_file = plot_roc_curve(visualization_results_dir, metrics_dict, alternative_colors, task_type)
    pr_image_file = plot_prc_curve(visualization_results_dir, metrics_dict, alternative_colors, task_type)
    image_files = [roc_image_file, pr_image_file]
    combined_image_file = os.path.join(visualization_results_dir, f'{task_type}_visualization.jpg')
    combined_image(image_files, combined_image_file)
    
    return combined_image_file

def plot_box(visualization_results_dir, metrics_dict, task_type):
    plt.rcParams['font.size'] = 12
    plt.rcParams['font.family'] = 'DeJavu Serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    box_width = 0.5

    results = {}
    results['pearson'], results['rmse'], results['mae'] = [], [], []
    for fold_result in metrics_dict['test'].values():
        results['pearson'].append(fold_result['corr'][0])
        results['rmse'].append(fold_result['rmse'])
        results['mae'].append(fold_result['mae'])
        
    sns.boxplot(data=results['pearson'], ax=axes[0], color='skyblue', width=box_width, showfliers=True)
    sns.swarmplot(data=results['pearson'], ax=axes[0], color='black', size=5, alpha=0.77)
    axes[0].set_title('10 folds Cross-Validation', fontsize=13)
    axes[0].set_ylabel("Pearson", fontsize=12)
    axes[0].grid(False)
    axes[0].set_xticks([])
    axes[0].axhline(y=np.median(results['pearson']), xmin=0, xmax=0.25, color='gray', linestyle='--')
    axes[0].text(-0.37, np.median(results['pearson']), f"{np.median(results['pearson']):.3f}", fontsize=11, color='black', ha='center', va='bottom')

    sns.boxplot(data=results['rmse'], ax=axes[1], color='salmon', width=box_width, showfliers=True)
    sns.swarmplot(data=results['rmse'], ax=axes[1], color='black', size=5, alpha=0.77)
    axes[1].set_title('10 folds Cross-Validation', fontsize=13)
    axes[1].set_ylabel('RMSE', fontsize=12)
    axes[1].grid(False)
    axes[1].set_xticks([])
    axes[1].axhline(y=np.median(results['rmse']), xmin=0, xmax=0.25, color='gray', linestyle='--')
    axes[1].text(-0.37, np.median(results['rmse']), f"{np.median(results['rmse']):.4f}", fontsize=11, color='black', ha='center', va='bottom')

    sns.boxplot(data=results['mae'], ax=axes[2], color='lightgreen', width=box_width, showfliers=True)
    sns.swarmplot(data=results['mae'], ax=axes[2], color='black', size=5, alpha=0.77)
    axes[2].set_title('10 folds Cross-Validation', fontsize=13)
    axes[2].set_ylabel('MAE', fontsize=12)
    axes[2].grid(False)
    axes[2].set_xticks([])
    axes[2].axhline(y=np.median(results['mae']), xmin=0, xmax=0.25, color='gray', linestyle='--')
    axes[2].text(-0.37, np.median(results['mae']), f"{np.median(results['mae']):.3f}", fontsize=11, color='black', ha='center', va='bottom')
    
    fig.suptitle(task_type.title(), fontsize=14, y=0.97)
    plt.tight_layout()
    image_file = os.path.join(visualization_results_dir, f'{task_type}_visualization.jpg')
    plt.savefig(image_file, bbox_inches='tight', dpi=1024)

    return image_file

def Visualize(metrics_file_path, task_type):
    task_type = task_type.lower()
    visualization_results_dir = "visualization_results"
    os.makedirs(visualization_results_dir, exist_ok=True)
    metrics_dict = json.load(open(metrics_file_path, 'r'))
    if task_type in ['stability', 'fluorescence']:
        set_colors = {'Train set':'red','Valid set': 'slategrey', 'Test set': 'steelblue'}
        visualization_results = plot_reg(visualization_results_dir, metrics_dict, set_colors, task_type, min_value=-2, max_value=5, interval=1)
    elif task_type in ['pdbbind']:
        set_colors = {'Train set': 'blueviolet', 'Valid set': '#5d3954', 'Test set (core2016)': 'darkcyan'}
        visualization_results = plot_reg(visualization_results_dir, metrics_dict, set_colors, task_type, min_value=-0, max_value=16, interval=5)
    elif ('remote' in task_type or 'homology' in task_type) or ('secondary' in task_type or 'structure' in task_type):
        alternative_colors = ['#d4af37', '#8a9a5b', '#536878']
        visualization_results = plot_cls(visualization_results_dir, metrics_dict, alternative_colors, task_type)
    elif task_type in ['kinase'] or ('antigen' in task_type or 'binding' in task_type):
        alternative_colors = ['navy', 'r', 'b']
        visualization_results = plot_binarycls(visualization_results_dir, metrics_dict, alternative_colors, task_type)
    elif task_type in ['skempi']:
        visualization_results = plot_box(visualization_results_dir, metrics_dict, task_type)
        
    return visualization_results

### OpenAI key

In [ ]:
config_list = [{ 'model': 'gpt-4-32k', 'api_key': 'openai-key'}] 

manager_llm_config = {
    "config_list": config_list,
    "seed": 42,
}

llm_config = {
    "seed": 42,
    "config_list": config_list,
    "functions": [
        {
            "name": "Analyze_Protein_Property",
            "description": "Analyze the protein properties including stability, fluoresence, remote homology, secondary structure and so on. Finally return the file path of task results",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The file path of protein data used to analyze protein properties"
                    },
                    "task_type": {
                        "type": "string",
                        "description": "The task name"
                    },
                    "finetune": {
                        "type": "boolean",
                        "description": "Is fine-tuning needed for the protein task? If the task instructions do not mention finetuning, the default setting is boolean false.",
                        "default": False
                    }
                }
            },
            "required": ["file_path", "task_type", "finetune"]
        },
        {
            "name": "Analyze_Protein_Drug_Interaction", 
            "description": "Analyze the interaction of protein and drug including pdbbind, kinase and so on. Finally return the file path of task results",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The file path of protein data used to analyze the interaction of protein and drug"
                    },
                    "task_type": {
                        "type": "string",
                        "description": "The task name"
                    },
                    "finetune": {
                        "type": "boolean",
                        "description": "Is fine-tuning needed for the protein task? If the task instructions do not mention finetuning, the default setting is boolean false.",
                        "default": False
                    }
                }
            },
            "required": ["file_path", "task_type", "finetune"]
        },
        {
            "name": "Analyze_Protein_Protein_Interaction", 
            "description": "Analyze the interaction of protein and protein including skempi, etc. Finally return the file path of task results",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The file path of protein data used to analyze the interaction of protein and protein"
                    },
                    "task_type": {
                        "type": "string",
                        "description": "The task name"
                    },
                    "finetune": {
                        "type": "boolean",
                        "description": "Is fine-tuning needed for the protein task? If the task instructions do not mention finetuning, the default setting is boolean false.",
                        "default": False
                    }
                }
            },
            "required": ["file_path", "task_type", "finetune"]
        },
        {
            "name": "Evaluate",
            "description": "Calculate the metrics of the returned inference results by the tasks including analyzing protein property, analyzing protein-Drug interaction and analyzing protein-protein interaction. Finally return the metrics",
            "parameters": {
                "type": "object",
                "properties": {
                "inference_file_path": {
                        "type": "string",
                        "description": "The path of returned inference results by the tasks"
                    },
                    "task_type": {
                        "type": "string",
                        "description": "The task name"
                    }
                }
            },
            "required": ["inference_file_path", "task_type"]
        },
        {
            "name": "Visualize",
            "description": "Visualize by the metrics and save it as a jpg",
            "parameters": {
                "type": "object",
                "properties": {
                    "metrics_file_path": {
                        "type": "string",
                        "description": "The path of returned metrics results by the tasks"
                    },
                   "task_type": {
                        "type": "string",
                        "description": "The task name"
                    }
                }
            },
            "required": ["metrics_file_path", "task_type"]
        }
    ]
}

### Agents

In [ ]:
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config={"work_dir": ".", "use_docker": False},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
                    Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

inference_agent = autogen.AssistantAgent(
    name="Inference",
    llm_config=llm_config,
    function_map={"Analyze_Protein_Property": Analyze_Protein_Property,
                  "Analyze_Protein_Drug_Interaction": Analyze_Protein_Drug_Interaction,
                  "Analyze_Protein_Protein_Interaction": Analyze_Protein_Protein_Interaction
                  }
)

metrics_agent = autogen.AssistantAgent(
    name="Evaluation",
    llm_config=llm_config,
    function_map={"Evaluate": Evaluate}
)

draw_agent = autogen.AssistantAgent(
    name="Visualization",
    llm_config=llm_config,
    function_map={"Visualize": Visualize}
)

In [ ]:
groupchat = autogen.GroupChat(agents=[user_proxy, inference_agent, metrics_agent, draw_agent], messages=[],max_round=150) # 群聊 最大的chat轮次12次
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=manager_llm_config)  #管理员，可以与之交互

### WO Finetune

* property

In [ ]:
user_proxy.initiate_chat(manager, message="""Can you analyze on stability task which belongs to protein property prediction benchmark, where the data file and format are as follows, and evaluate the predictions based on the task, finally visualize the evaluation results?

../downstream_task_small/stability/sequence_go.txt

Start the work now.
""")

* PDI

In [ ]:
user_proxy.initiate_chat(manager, message="""Can you analyze on kinase task which belongs to protein-drug interaction prediction benchmark, where the data file and format are as follows, and evaluate the predictions based on the task, finally visualize the evaluation results?

downstream_task/kinase/samples_seq_mole_go.txt

Start the work now.
""")

### With Finetune

* property

In [ ]:
user_proxy.initiate_chat(manager, message="""Can you fine-tune and analyze on antigen binding task which belongs to protein property prediction benchmark, where the data file and format are as follows, and evaluate the predictions based on the task, finally visualize the evaluation results?

downstream_task/antigen_binding/sequence_go.txt

Start the work now.
""")